<br><h1>[<span style="color:red">COVID-19</span> Analysis & Visualization] <span style="color:red">뭉치면 </span>죽고 퍼지면 산다! </h1></br> 

안녕하세요. Mulcamer팀입니다. 
Mulcamer(Multicampus+Learner)는 현재 삼성 멀티캠퍼스에서 데이터 분석과 AI과정을 공부하고 있는 학생들로 이루어진 팀입니다. 현재 가장 큰 이슈인 코로나라는 펜데믹(pandemic)에 대응하여 인류 전체의 문제를 해결하기 위한 노력과 지식 공유를 목적으로 프로젝트에 임하였습니다. 더 나아가 저희팀의 분석 자료를 통해 앞으로의 코로나 관련 정책 방향성이나 사람들 인식의 변화에 영향을 줄 수 있는 객관적인 저널이 되고자 하는 목표가 있습니다.

우선, 진행방향을 잡기위해 전체적인 데이터에 대한 EDA를 진행하였습니다. 그 후 EDA를 바탕으로 총 4가지의 주제를 잡고 분석 작업을 진행하였습니다. 코로나 바이러스를 예방하기 위해서는 현재 <span style="color:red">감염률이 높은 집단</span>과 <span style="color:red">사망률이 높은 집단</span>에 대한 현황 및 원인을 제대로 파악해야 한다고 생각하였습니다. 이를 통해 각별한 주의가 필요한 집단 스스로 인지하고 정부는 이에 대한 적절한 대처 방안을 세워야 코로나 바이러스로부터 1차적인 예방을 할 수 있습니다. 
###(서두 더 조금 더 수정)


# 전체 결론 핵심 요약

    이번 분석을 통해 무엇보다 집단감염으로 인한 큰 영향과 위험성을 파악하였으며, 이는 백신개발 및 전세계적으로 완전한 종식이 될때까지 집단감염에 유의하여야 합니다. <부록>에서  최근 연휴(4월말~5월초) 기간의 사람들의 활동성이 늘었다는 것을 확인하였습니다. 사회적 거리두기가 끝나지 않은 상태에서도 경각심이 줄어들었다는 것을 알 수 있습니다. 이는 다시 신천지 전후와 같이 급격하게 확진자수가 증가하는 최악의 집단감염 사태를 다시 초래할 수 있습니다. 그러므로 집단감염에 대한 각별한 대책과 경각심을 위한 정책이 한번 더 시행될 필요성이 있다고 생각합니다.

---

## 다른 포스팅 
#### [Tableau & Plotly 활용 전반적 EDA 와 시각화](https://dacon.io/competitions/official/235590/codeshare/1032?page=1&dtype=recent)
#### [Topic_1 : 20대 감염원인과 성향 파악](https://dacon.io/competitions/official/235590/codeshare/1069?page=1&dtype=recent)
#### [Topic_2 : 과연, COVID19는 누구에게 치명적일까?](https://dacon.io/competitions/official/235590/codeshare/1067?page=1&dtype=recent)
#### [Topic_3 : 수도권과 비수도권 확진자 양상의 차이?](https://dacon.io/competitions/official/235590/codeshare/1068?page=1&dtype=recent)

---

In [1]:
# essential libraries
import os
import math
import numpy as np
import pandas as pd
import datetime as dt

# prediction model
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from PIL import Image

# API, URL, DB
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlencode
from urllib.parse import quote_plus
from urllib.request import urlopen
import sqlite3




# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

#pandas와 같이 plotly에서도 간단한 시각화 지원
import cufflinks as cf
cf.go_offline(connected=True)

# for offline ploting
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
print(px.colors.qualitative.Prism)

['rgb(95, 70, 144)', 'rgb(29, 105, 150)', 'rgb(56, 166, 165)', 'rgb(15, 133, 84)', 'rgb(115, 175, 72)', 'rgb(237, 173, 8)', 'rgb(225, 124, 5)', 'rgb(204, 80, 62)', 'rgb(148, 52, 110)', 'rgb(111, 64, 112)', 'rgb(102, 102, 102)']


In [3]:
# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['#636EFA', '#EF553B', 
                       '#00CC96', '#AB63FA',
                       '#FFA15A', '#19D3F3',
                       '#FF6692', '#B6E880',
                       '#FF97FF', '#FECB52','rgb(57,105,172)']

DEFAULT_PLOTLY_COLORS2 = ['rgb(230,210,0)', 'rgb(241,105,19)','rgb(8,104,172)', 'rgb(0,109,44)', 'rgb(131,75,160)']
DEFAULT_PLOTLY_COLORS2_2 = ['rgb(230,210,0)', 'rgb(241,105,19)', 'rgb(241,105,19)', 'rgb(8,104,172)', 'rgb(8,104,172)', 'rgb(0,109,44)','rgb(0,109,44)', 'rgb(131,75,160)', 'rgb(131,75,160)', 'rgb(131,75,160)']

layout_setting = {'xaxis_title':'Date',
                  'yaxis_title':'Number',
                  'font':dict(size=18,color='#60606e',
                              family='Franklin Gothic')}

layout_font = {'font':dict(size=18,color='#60606e',
                           family='Franklin Gothic' )}

In [4]:
path = '../covid'
os.chdir('./data')
file_name_list = os.listdir()

In [5]:
file_name_list

['avgAgeProvince.jpg',
 'Case.csv',
 'PatientInfo.csv',
 'PatientRoute.csv',
 'PatientTrend_20200414_20200427.xlsx',
 'Region.csv',
 'SearchTrend.csv',
 'SeoulFloating.csv',
 'Time.csv',
 'TimeAge.csv',
 'TimeGender.csv',
 'TimeProvince.csv',
 'Weather.csv',
 'WHO-COVID-19-global-data.csv']

In [7]:
case = pd.read_csv("Case.csv")
region = pd.read_csv("Region.csv")
patientinfo = pd.read_csv("PatientInfo.csv")
route = pd.read_csv( 'PatientRoute.csv')
timeProve = pd.read_csv("TimeProvince.csv")
timeAge = pd.read_csv("TimeAge.csv")
timeGender = pd.read_csv("TimeGender.csv")
seoulFloating = pd.read_csv('SeoulFloating.csv')
# population = pd.read_csv("202003Korea_Population.csv", encoding='ANSI') # 2020.03 한국 인구수 from 통계청
# numProvince = pd.read_csv("20200428ncovProvince.csv", index_col=False) # 2020.04.28 질병관리본부데이터
# http://ncov.mohw.go.kr/tcmBoardView.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=354256&contSeq=354256&board_id=140&gubun=BDJ

---
# Topic 4. Logistic Function과 Linear Regression을 통한 확진자 수 예측

누적 확진자 수의 양상을 파악하고 예측하기 위해서 로지스틱 함수와 선형회귀 분석을 통해서 확진자 수를 확인해보고 예측을 진행하였습니다. 대표적인 한국, 중국, 이탈리아, 미국의 누적 확진자 수의 방향을 예측해보았습니다.

더 나아가 신천지 집단 감염이후 급격하게 증가하는 확진자수가 증가하는 것을 고려하여, 만약 신천지 사태가 발생하지 않았다면 한국의 확진자 수가 어떻게 되었을지 예측해보았습니다.

## 결론 및 요약

로지스틱 함수와 선형회귀를 통해 확진자 증가 양상과 종식을 예측해 보았습니다. 그 중 특징적인 4개의 국가의 코로나 확진자 추세를 예측해보았고, 더 나아가 신천지 집단 감염이 없었을 경우를 예측해보았습니다.

### 1. 중국, 한국, 이탈리아, 미국 확진자 수 예측하기
현재 한국과 중국은 소강상태에 가깝고 종식으로 이어질 것으로 예측할 수 있습니다. 반면, 이탈리아와 미국은 여전히 확진자가 증가하는 방향임을 예측할 수 있습니다. 

### 2.신천지 집단 감염이 일어나지 않았다면?
<span style="color:red">신천지 이전 기간을 통해 예측한 결과 확진자가 거의 증가하지 않는 소강 상태로 갈 것이라고 예측</span>이 되었습니다. 하지만 코로나 집단 감염 발생으로 완전히 다른 그래프 양상을 보이며 급격히 증가하였습니다. 이를 보았을때 신천지 대규모 감염이 발생하지 않았다면 이렇게 많은 확진자가 발생하지 않았을 것이며, 한국은 지금보다 일찍 코로나 종식에 가까워졌을 것이라 예상할 수 있습니다.


따라서, <span style="color:red">신천지 사태와 같은 예상치 못한 대규모 집단 감염이 발생하지 않도록 완전 종식까지 긴장을 늦추지 않고 경계</span>해야합니다. 한 확진자가 대규모 감염을 발생시킬 수 있으며, 이는 곧 전국적인 확산으로 퍼질 수 있습니다. 코로나 증가추세가 꺽인 현재의 한국과 중국은 코로나 종식까지 철저한 위생관리 및 방역활동을 진행해야하며, 사회적 거리두기를 지속해야 합니다. 사회적 거리두기가 끝나더라도 <span style="color:red">완전 종식이 될 때까지, 단체활동을 줄이고 불필요한 접촉 및 외출을 최소화</span>해야합니다. 

## 1. 모델 설명 및 선정 이유

In [8]:
# 기본 설정
WHO_Data = pd.read_csv('WHO-COVID-19-global-data.csv', header=0,
                      names=['Date', 'Country Code', 'Country', 'Region', 'Daily Deaths', 'Deaths', 'Daily Confirmed', 'Confirmed'])
WHO_Data['Date'] = WHO_Data['Date'].apply(pd.Timestamp)
WHO_Data.drop(['Country Code', 'Region'], axis=1, inplace=True)

#### 예측 방향
앞서 분석했던 방향에 맞추어 더 자세한 예측(감염 경로 예측, 사망 원인 예측, 기저질환 예측 등)을 진행하고자 하였으나, 급격한 전염병 증가로 인하여 자세하게 정확하게 정리된 데이터를 구하기가 어려었습니다. 게다가, 결측치나 기타로 분류된 파악 불가능한 데이터들도 많아 한계가 많아 예측의 정확성 및 근거가 부족할 것이라고 판단했습니다. 그리하여, 결측치가 없고 가장 정리가 잘 된 확진자 수 데이터를 통해서 앞으로의 증가 방향성을 예측해 보기로 결정하였습니다.

#### 로지스틱 그래프 설명
기본적인 logistic curve의 모양은 sigmoid curve 모양과 같고 수식은 다음과 같습니다.

$$f (x) = \frac{L}{1+e^{-k(x-x_0)}}$$

자연 발생의 증가는 지수함수적으로(exponentialy)증가하지만, 실제 현실에서는 최대 증가치가 정해져 있으므로 특정 값으로 수렴하게됩니다. 이러한 특성을 로지스틱 함수(Logistic Curve)가 이를 잘 나타냅니다. 실제로 많은 감염병 예측이나 자연발생 증가에 대한 연구에 많이 사용되는 방식입니다. 그리하여 확진자 증가 방향 예측에 유용할 것이라 판단했습니다.

로지스틱 그래프의 특성상 최종적으로는 특정 값에 수렴하는 형태를 가집니다. 그리하여 아래 그래프와 같이 예측한 확진자 수 그래프와 실제 확진자 수 그래프가 같이 수평한 모습을 보이며 수렴해 간다면, 확진자 수가 거의 나오지 않는 소강상태에 접어든 것으로 판단할 수 있습니다. 예측과 다르게 수렴하지 않고 실제값이 위로 향하고 있다면 아직 확진자가 증가하는 추세로 확인할 수 있습니다.

로지스틱 함수의 초기값 L, k, x0은 curve fit을 통해서 적절한 값을 다시 찾을 것이므로 다음과 같이 대략적으로 값을 설정하였습니다. 
2월 18일 신천지 31번 확진자가 발생하기 이전인 2월 17일 이전의 중국 확진자 데이터를 바탕으로 로지스틱 함수의 파라미터를 설정하였습니다.

L : 최대 확진자수 : 17일 전까지의 중국 확진자 수 사용

k : Growth Rate : 중국과 세계의 Growth Rate 평균 값 사용

중국은 이제 급격히 증가하는 단계이고 다른 세계 국가들은 이제 초기 단계이므로 평균값을 사용하였습니다.

x0 : mid point : 최대 확진자수에 이르기 까지의 중간 지점

#### 선형회귀 그래프 설명
전체적인 확진자 수에대해 log를 취해서 전반적인 증가 추세를 선형회귀로 확인할 수 있습니다. 하지만, 저희는 최근 n일 간의 확진자 수를 통해서 선형회귀 선의 기울기를 통해서 최근 n일 동안의 확진자 증가 트렌드와 증가 속도를 확인하였습니다. n일 까지의 선형회귀 선의 기울기 보다 n일 이후의 데이터가 낮은 기울기를 가지는 모습을 보인다면 점차 확진자수 증가세가 줄어든 것을 의미합니다. 반대로 큰 값을 가지는 경우 더욱 빠르게 확진자 수가 증가하는 것을 의미합니다. 

로지스틱 함수를 따라 줄어드는 모습을 보인다면 점차 소강상태에 가까워지는 것을 의미하며, 선형회귀 선을 따라 증가하면, 증가추세가 이어지고 있는 것을 의미합니다. 만일 선형회귀 선보다 더 가파른 기울기를 보인다면, 코로나 신규 확진자 증가수가 급격히 늘어난 것으로 확산이 계속 증가될 것으로 전망할 수 있습니다.

In [9]:
# 로지스틱 함수
def logistic(x, L, k, x0):
    return L / (1 + np.exp(-k * (x - x0)))

# Growth Rate 구하기 위한 함수
def add_rates(df):
    df['Mortality Rate'] = df['Deaths'] / df['Confirmed']
    df['Growth Rate'] = df['Confirmed'] / df['Confirmed'].shift(1) - 1 # 당일 확진자 / 전일 확진자 - 1
    df['Growth Rate Change'] = df['Growth Rate'].diff().shift(-1)
    return df

In [10]:
# 신천지 기간 전 세계 확진자 Growth Rate
BeforeSCJ = WHO_Data[WHO_Data['Date'] < "2020-02-17"] # 신천지 이전 기간 데이터
GR_sum, cnt = 0, 0 
for country in BeforeSCJ['Country'].unique():
    df = BeforeSCJ[BeforeSCJ['Country'] == country]
    if len(df) != 0: # 신천지 기간 이전 데이터가 없는 경우 제외
        GR = add_rates(df)['Growth Rate'].mean()
        if GR != 0:  # 데이터는 있으나 코로나 시작 전인 경우 제외
            GR_sum += GR
            cnt += 1 # 코로나가 발생한 country 수
GR_mean = GR_sum / cnt # 약 13 %

In [11]:
china = WHO_Data[WHO_Data['Country'] == 'China']
ChinaBeforeSCJ = china[china['Date'] < "2020-02-17"] # 신천지 이전 기간 데이터
ChinaBeforeSCJ = add_rates(ChinaBeforeSCJ)

GR_China = ChinaBeforeSCJ['Growth Rate'].mean() # 약 0.24

In [12]:
k = np.round(np.mean([GR_mean,GR_China]),2) # 세계와 중국의 평균 약 0.19
L = ChinaBeforeSCJ['Confirmed'].max() # 신천지 전 기간 가장 최근 누적 확진자 수
x0 = (ChinaBeforeSCJ['Confirmed'] <= ChinaBeforeSCJ['Confirmed'].max() / 2 + 1).sum() # mid point 까지의 일 수
p0 = [L, k, x0]
p0

[68584, 0.19, 28]

In [13]:
DAYS2FIT = 7
PREDICT_PERIOD = 7

# 선형회귀 모델 함수
def linearRgPrediction(df, Days2Fit, Days2Pred):
    lastData = df[-Days2Fit:] # 최근 n 일 데이터
    fittingDate = lastData['Date']
    
    x = np.array(range(len(lastData))).reshape(-1,1)
    y = lastData['Confirmed'].to_numpy().reshape(-1,1) # n일의 확진자 수

    linearReg = LinearRegression().fit(x,y) # 선형 회귀
    predRange = pd.date_range(fittingDate.iloc[0], # 예측 기간 Days2Fit~Days2Pred까지
                              fittingDate.iloc[-1] + dt.timedelta(days=Days2Pred))
    predX = np.array(range(len(predRange))).reshape(-1,1)
    predY = linearReg.predict(predX)
    
    return predRange, predY

In [14]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['<b>로지스틱 : 소강 트렌드</b>', '<b>로지스틱 : 증가 트렌드</b>',
                                                   '<b>선형회귀 : 소강 트렌드</b>', '<b>선형회귀 : 확산 트렌드</b>'])
# 중국 
china = WHO_Data[WHO_Data['Country'] == 'China']
popt, pcov = curve_fit(logistic, range(len(china)), china['Confirmed'], p0)

predChina = logistic(range(len(china)+PREDICT_PERIOD), *popt)


# 중국 확진자 수 그래프
# 두 그래프 차이를 비교하기 위한 것으로 최대 값으로 나눠 0~1사이로 나타내도록함
fig.add_trace(go.Scatter(x=list(range(len(china))), 
                         y=china['Confirmed']/china['Confirmed'].values[-1],
                         marker=dict(color = DEFAULT_PLOTLY_COLORS[0]), showlegend=False),row=1,col=1)
# 로지스틱 확진자 그래프 
fig.add_trace(go.Scatter(x=list(range(len(china))), 
                         y=(predChina/predChina[-1])[:-PREDICT_PERIOD],
                         marker=dict(color = DEFAULT_PLOTLY_COLORS[1]), showlegend=False),row=1,col=1)
# 로지스틱 예측 기간 그래프
fig.add_trace(go.Scatter(x=list(range(len(china)-1,len(china)+PREDICT_PERIOD)), 
                         y=(predChina/predChina[-1])[-PREDICT_PERIOD-1:], mode='lines', 
                         line = dict(color=DEFAULT_PLOTLY_COLORS[4], 
                         width=4, dash='dot'), showlegend=False),row=1,col=1)


us = WHO_Data[WHO_Data['Country'] == 'United States of America'] # WHO 데이터
popt, pcov = curve_fit(logistic, range(len(us)), us['Confirmed'], p0)
predUS = logistic(range(len(us)+PREDICT_PERIOD), *popt)

# 기존 확진자 그래프
fig.add_trace(go.Scatter(x=list(range(len(us))), 
                         y=us['Confirmed']/us['Confirmed'].values[-1],
                         marker=dict(color = DEFAULT_PLOTLY_COLORS[0]),
                         name="Actual", showlegend=True),row=1,col=2)
# 로지스틱 함수 그래프 
fig.add_trace(go.Scatter(x=list(range(len(us))), 
                         y=(predUS/predUS[-1])[:-PREDICT_PERIOD],
                         marker=dict(color = DEFAULT_PLOTLY_COLORS[1]),
                         name='Logistic Curve', showlegend=True),row=1,col=2)
# 로지스틱 함수 예측 부분
fig.add_trace(go.Scatter(x=list(range(len(us)-1,len(china)+PREDICT_PERIOD)), 
                         y=(predUS/predUS[-1])[-PREDICT_PERIOD-1:], mode='lines', 
                         line = dict(color=DEFAULT_PLOTLY_COLORS[4], width=4, dash='dot'),
                         showlegend=True,name='Logistic Prediction'), row=1,col=2)

##### 선형회귀 #####
predDate, predY = linearRgPrediction(china, DAYS2FIT, PREDICT_PERIOD)

# 중국
# 기존 중국 확진자 그래프
fig.add_trace(go.Scatter(x=list(range(len(china))), 
                         y=china['Confirmed']/china['Confirmed'].values[-1],
                         marker=dict(color = DEFAULT_PLOTLY_COLORS[0]), showlegend=False),row=2,col=1)

# 최근 DAYS2FIT 기간의 데이터를 바탕으로 선형회귀 그래프 
# 중국 선형 회귀
fig.add_trace(go.Scatter(x=list(range(len(china)-PREDICT_PERIOD,len(china)+PREDICT_PERIOD)),
                         y=predY.reshape(1,-1)[0] / predY.reshape(1,-1)[0][:PREDICT_PERIOD].max(),
                         mode='lines', line = dict(color=DEFAULT_PLOTLY_COLORS[2], width=4, dash='dot'),
                         showlegend=False),row=2,col=1)

# 미국
predDate, predY = linearRgPrediction(us, DAYS2FIT, PREDICT_PERIOD)
fig.add_trace(go.Scatter(x=list(range(len(us))), 
                         y=us['Confirmed']/us['Confirmed'].values[-1],
                         marker=dict(color = DEFAULT_PLOTLY_COLORS[0]),
                         showlegend=False),row=2,col=2)

fig.add_trace(go.Scatter(x=list(range(len(us)-PREDICT_PERIOD,len(us)+PREDICT_PERIOD)),
                         y=predY.reshape(1,-1)[0] / predY.reshape(1,-1)[0][:PREDICT_PERIOD].max(),
                         line = dict(color=DEFAULT_PLOTLY_COLORS[2], width=4, dash='dot'),
                         mode='lines', showlegend=True, name='Linear prediction'),
                         row=2,col=2)

fig.update_layout(title='<b>코로나 확진자 추이 예측 모델 예시</b>',
                  height = 600, font = layout_setting['font'],
                  font_size=14, legend_orientation="h",
                  legend=dict( traceorder="normal", font=dict(family="sans-serif", size=15)))

fig.show()

## 2. 중국, 한국, 이탈리아, 미국 확진자 수 예측하기
로지스틱 함수와 선형회귀를 통해서 4개의 특징적인 국가의 확진자 수를 예측해보았습니다.
코로나 발생 근원지인 중국, 빠른 대처로 세계적 이목을 받고 있는 한국, 짧은 기간에 급격한 확산과 많은 사망자가 발생한 이탈리아,
확진자 수 세계 1위 미국, 이렇게 총 4개의 국가를 선정하였습니다.
중국과 한국은 소강 상태로 보이며, 이탈리아와 미국은 확산이 진행중인 상태로 판단하였습니다.

In [15]:
korea = WHO_Data[WHO_Data['Country'] == 'Republic of Korea']
korea = add_rates(korea)

koreaBeforeSCJ = korea[korea['Date'] < "2020-02-17"] # 신천지 이전 기간 데이터
koreaBeforeSCJ = add_rates(koreaBeforeSCJ)

italy = WHO_Data[WHO_Data['Country'] == 'Italy'] # WHO 데이터
italy = add_rates(italy)

us = WHO_Data[WHO_Data['Country'] == 'United States of America'] # WHO 데이터
us = add_rates(us)

china = WHO_Data[WHO_Data['Country'] == 'China']
china = add_rates(china)

In [16]:
################################## 4개 나라 그래프 #################################
DAYS2FIT = 5
PREDICT_PERIOD = 14

fig = make_subplots(rows=2, cols=2, subplot_titles=['<b>중국</b>', '<b>한국</b>','<b>이탈리아</b>', '<b>미국</b>'])
countries = [china, korea, italy, us]
name=['China','Korea','Italy','U.S.A']
for i in range(4):
    if i>1:
        row= 2
        col= i-1
    else:
        row=1
        col=i+1
    
    legend=False
    if i==3:
        legend=True
        
    # 날짜 설정
    actualDate = countries[i]['Date']
    predDate = pd.date_range(actualDate.iloc[0], actualDate.iloc[-1] + dt.timedelta(days=PREDICT_PERIOD))
    futureDate = pd.date_range(actualDate.iloc[-1], predDate[-1])

    # 선형회귀 예측
    predDate, predY = linearRgPrediction(countries[i], DAYS2FIT, PREDICT_PERIOD)
    # 로지스틱 함수 파라미터 찾기
    popt, pcov = curve_fit(logistic, range(len(countries[i])), countries[i]['Confirmed'], p0)
    
    # 실제 확진자 그래프
    fig.add_trace(go.Scatter(x=actualDate, y=countries[i]['Confirmed'],
                             marker=dict(color = DEFAULT_PLOTLY_COLORS[0]),
                             showlegend=legend, name='real'),
                             row=row,col=col)
    
    # Logistic Curve
    fig.add_trace(go.Scatter(x=actualDate,
                             y=logistic(range(len(countries[i])+PREDICT_PERIOD), *popt),
                             marker=dict(color = DEFAULT_PLOTLY_COLORS[1]),
                             showlegend=legend, name='Logistic Curve'),row=row,col=col)
    
    # Logistic Curve 예측 부분
    fig.add_trace(go.Scatter(x=futureDate, 
                             y=logistic(range(len(countries[i])+PREDICT_PERIOD), *popt)[-PREDICT_PERIOD-1:],
                             mode='lines', line = dict(color=DEFAULT_PLOTLY_COLORS[1], width=3, dash='dot'),
                             showlegend=legend, name='Logistic Prediction'),row=row,col=col)

    # 선형 회귀 선
    fig.add_trace(go.Scatter(x=predDate, y=predY.reshape(1,-1)[0], mode='lines',
                             line = dict(color=DEFAULT_PLOTLY_COLORS[2], width=3, dash='dot'),
                             showlegend=legend, name='Linear regresstion prediction'),
                             row=row,col=col)
    
fig.update_layout(title='<b>선형회귀와 로지스틱 함수를 사용한 확진자 수 방향 예측</b>',
                  font = layout_setting['font'],
                  font_size=14, legend_orientation="h",
                  legend=dict( traceorder="normal", font=dict(family="sans-serif", size=15)))
fig.show()

중국과 한국은 로지스틱 그래프와 수렴해가는 모습을 보이고 있습니다. 또한, 선형회귀식의 기울기가 매우 낮아졌으며 로지스틱 함수와 거의 평행해지는 것을 확인할 수 있습니다. 이를 통해서 한국과 중국은 코로나 종식에 가까워지고 있는 모습을 확인할 수 있습니다. 

이탈리아와 미국은 아래 그래프에서 더욱 자세하게 보겠습니다.

In [17]:
################################## 4개 나라 그래프 #################################
fig = make_subplots(rows=1, cols=2, subplot_titles=['<b>이탈리아</b>', '<b>미국</b>'])
countries = [italy, us]
name=['Italy','U.S.A']
legend = False
for i in range(2):
    row, col = 1, i+1
    if col == 2:
        legend = True
    
    # 날짜 설정
    actualDate = countries[i]['Date'][-DAYS2FIT:]
    predDate = pd.date_range(actualDate.iloc[0], actualDate.iloc[-1] + dt.timedelta(days=PREDICT_PERIOD))
    futureDate = pd.date_range(actualDate.iloc[-1], predDate[-1])
    
    # 선형회귀
    predDate, predY = linearRgPrediction(countries[i], DAYS2FIT, PREDICT_PERIOD)
    # 로지스틱 함수
    popt, pcov = curve_fit(logistic, range(len(countries[i])), countries[i]['Confirmed'], p0)
    
    # 실제 확진자 그래프
    fig.add_trace(go.Scatter(x=actualDate, y=countries[i]['Confirmed'][-DAYS2FIT:],
                             marker=dict(color = DEFAULT_PLOTLY_COLORS[0]),
                             showlegend=legend, name='real'),
                             row=row,col=col)
    
    # Logistic Curve
    fig.add_trace(go.Scatter(x=actualDate,
                             y=logistic(range(len(countries[i])+PREDICT_PERIOD), *popt)[-DAYS2FIT-PREDICT_PERIOD:-PREDICT_PERIOD],
                             marker=dict(color = DEFAULT_PLOTLY_COLORS[1]),
                             showlegend=legend, name='logistic curve'),row=row,col=col)
    
    # Logistic Curve 예측 부분
    fig.add_trace(go.Scatter(x=futureDate, 
                             y=logistic(range(len(countries[i])+PREDICT_PERIOD), *popt)[-PREDICT_PERIOD-1:], mode='lines',
                             line = dict(color=DEFAULT_PLOTLY_COLORS[1], width=3, dash='dot' ),
                             showlegend=legend, name='logistic prediction'),row=row,col=col)
    
    # 선형 회귀 선
    fig.add_trace(go.Scatter(x=predDate, y=predY.reshape(1,-1)[0], mode='lines',
                             line = dict(color=DEFAULT_PLOTLY_COLORS[2], width=3, dash='dot'),
                             showlegend=legend, name='linear regresstion prediction'),
                             row=row,col=col)
    
fig.update_layout(title='<b>최근 5일을 통한, 이후 진행 방향 예측</b>',
                  font = layout_setting['font'],
                  font_size=14, legend_orientation="h",
                  legend=dict( traceorder="normal", font=dict(family="sans-serif", size=15)))
fig.show()

이탈리아와 미국은 로지스틱과 선형 그래프의 진행방향에 차이가 꽤 있는 것을 확인할 수 있습니다. 로지스틱 함수의 방향과 다른 방향으로 진행되고 있는 확진자수 증가 추이와 선형회귀 식의 기울기를 통해서 여전히 확진자가 증가하는 추세임을 예상할 수 있습니다. 미국과 이탈리아는 더욱 확진 방지를 위한 노력이 필요할 것입니다.

## 3. 신천지 집단 감염이 일어나지 않았다면?

Topic 1과 Topic 3를 통해서 집단 감염이 가장 확진자를 많이 발생시키는 것임을 확인하였습니다. 이를 기반으로 우리나라의 가장 많은 확진자를 발생시킨 집단 감염 사태인 신천지 집단 감염이 만약 발생하지 않았다면 확진자 수가 어떻게 변화되었을지를 확인해보고자 하였습니다. 그리하여 신천지 첫 확진자 발생일 2월 17일을 기준으로 그 전까지의 데이터를 바탕으로 예측을 진행하고 비교해보았습니다.

In [18]:
DAYS2FIT = 5
PREDICT_PERIOD = 7
predDatelinear, predY = linearRgPrediction(koreaBeforeSCJ, DAYS2FIT, PREDICT_PERIOD)

popt, pcov = curve_fit(logistic, range(len(koreaBeforeSCJ)), koreaBeforeSCJ['Confirmed'], p0)
predLogistic = logistic(range(len(koreaBeforeSCJ)+DAYS2FIT), *popt)

actualDate = koreaBeforeSCJ['Date']
predDate = pd.date_range(actualDate.iloc[0], actualDate.iloc[-1] + dt.timedelta(days=DAYS2FIT))
futureDate = pd.date_range(actualDate.iloc[-1], predDate[-1])

fig = go.Figure()
fig.add_trace(go.Scatter(x=actualDate, y=koreaBeforeSCJ['Confirmed'],
              marker=dict(color = DEFAULT_PLOTLY_COLORS[0]),name='Confirmed'))
fig.add_trace(go.Scatter(x=actualDate, y=predLogistic[:-DAYS2FIT], # 입력데이터 까지의 예측값
        marker=dict(color = DEFAULT_PLOTLY_COLORS[1]),name='Logistic prediction'))
fig.add_trace(go.Scatter(x=futureDate, y=predLogistic[-DAYS2FIT:], # 입력이후 진행방향
                         line = dict(color=DEFAULT_PLOTLY_COLORS[1], width=1, dash='dot'),
                         showlegend=True,name='Logistic Prediction Prediction'))

fig.add_trace(go.Scatter(x=predDatelinear, y=predY.reshape(1,-1)[0],
                         line = dict(color=DEFAULT_PLOTLY_COLORS[2], width=3, ),#dash='dash'
                         showlegend=True, name='Linear regresstion Prediction'))

fig.update_layout(title='<b>신천지 발생 이전 확진자 수 및 예측</b>',
                  font = layout_setting['font'],
                  font_size=14, legend_orientation="h",
                  legend=dict( traceorder="normal", 
                             font=dict(family="sans-serif", size=15))) 
fig.show()

신천지 이전 까지의 데이터를 통해서 예측 그래프를 그려보았습니다. 로지스틱 그래프는 점차 소강상태에 접어들기 시작하였으며, 선형회귀 선의 기울기가 상당히 낮아진 것을 확인할 수 있습니다. 두 그래프가 점차 평행해 나가는 것을 보며 어느정도 코로나가 확산이 줄어들고 <span style="color:red">소강상태로 진입하는 모습을 예상</span>해 볼 수 있습니다.

In [19]:
lastData = korea[-DAYS2FIT:]
predDate, predY = linearRgPrediction(koreaBeforeSCJ, 10, PREDICT_PERIOD)
popt, pcov = curve_fit(logistic, range(len(koreaBeforeSCJ)), koreaBeforeSCJ['Confirmed'], p0)

logisticPeriod = pd.date_range(koreaBeforeSCJ['Date'].iloc[0], 
                               koreaBeforeSCJ['Date'].iloc[-1] + dt.timedelta(days=PREDICT_PERIOD))

fig = go.Figure()

fig.add_trace(go.Scatter(x=koreaBeforeSCJ['Date'], y=koreaBeforeSCJ['Confirmed'],
              marker=dict(color = DEFAULT_PLOTLY_COLORS[0]),name='confirmed'))

fig.add_trace(go.Scatter(x=predDate[-9:], y=predY.reshape(1,-1)[0][-9:],
        line = dict(color=DEFAULT_PLOTLY_COLORS[2], width=3, dash='dot'),name='linear regresstion prediction'))

fig.add_trace(go.Scatter(x=korea['Date'][27:33], y=korea['Confirmed'][27:33],
        line = dict(color=DEFAULT_PLOTLY_COLORS[3], width=3, dash='dot'),name='Actual Confirmed'))

fig.add_trace(go.Scatter(x=logisticPeriod[-9:], y=logistic(range(len(koreaBeforeSCJ)+PREDICT_PERIOD), *popt)[-9:],
        line = dict(color=DEFAULT_PLOTLY_COLORS[1], width=3, dash='dot'),name='logistic prediction'))

# fig.add_trace(go.Scatter(x=list(range(len(koreaBeforeSCJ),len(koreaBeforeSCJ)+7)), y=logistic(range(len(koreaBeforeSCJ)+7), *popt)[-7:],
#                          line = dict(color=DEFAULT_PLOTLY_COLORS[1], width=1, dash='dot'),name='prediction'))
fig.update_layout(title='<b>신천지 확진 이후 3일</b>',
                  font = layout_setting['font'],
                  height=500, font_size=14, legend_orientation="h",
                  legend=dict( traceorder="normal", 
                             font=dict(family="sans-serif", size=15))) 


2월 17일 이후 3일을 추가하여 집단감염 발생으로 늘어난 확진자 수의 모습을 확인해보았습니다.
신천지 발생 이전에 예측했던 <span style="color:red">선형회귀 선을 뚫고 수직에 가까운 기울기를 보이며 확진자가 증가</span>하고 있습니다. 
자연적인 접촉으로 확산되는 지수적 증가와 상당히 차이가 있으며, 이전까지의 전체적인 트렌를 깨고 새로운 양상을 보여주고 있습니다. 
2월 16일 까지의 전체 확진자가 30명이었던 반면, 2월 19일 하루에 20명, 20일 하루에 53명의 대량 신규 확진자가 발생하였습니다. 

신천지 확진자 발생 전까지는 큰 확진 없이 미미하게 진행되었고 예측 그래프를 통해서 점차 소강 상태로 접어드는 단계였음을 확인하였습니다. 하지만, 신천지 대규모 확진으로 인하여 우리나라의 코로나 확진양상이 급격하게 바뀌었으며, 대규모 집단 감염의 위험성을 가시적으로 확인할 수 있습니다.

# 최종 결론

<span style="color:red">신천지 확진자 발생 전까지는 큰 확진 없이 미미하게 진행</span>되었고 예측 그래프를 통해서 점차 소강 상태로 접어드는 단계였음을 확인하였습니다. 하지만, 신천지 대규모 확진으로 인하여 우리나라의 코로나 확진양상이 급격하게 바뀌었으며, 대규모 집단 감염의 위험성을 가시적으로 확인할 수 있습니다.